In [ ]:
!pip install pymupdf
!pip install TTS==0.14.0
!pip install pydub

In [ ]:
import fitz
import re
from TTS.api import TTS
from pydub import AudioSegment
import time
import nltk
nltk.download('punkt')  # Download the necessary resource for tokenization


voice = "/content/w.wav"
pdf_file_path = "/content/My Inventions Nikola Tesla.pdf"


G = True
clone = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=G)

language = "en"
output_audio_files = []  # To store paths of individual audio files
contents_to_read = []  # To store contents of the table of contents
default_texts = [
        "The audio you are currently experiencing has been generated using proprietary technology from Tesla dot AI and Coqui, licensed solely for non-commercial purposes. Any unlawful use or distribution of this audio, including its incorporation into illegal activities such as pirated books or any other illegal content, is strictly prohibited and may result in legal consequences.",  # Non-commercial license statement
        "Blind Owl",  # Replace with the book's name
        "by Sadegh Hedayat",  # Replace with the author's name
        ]
def Zero_Shot_CloneTTS(voice, language, text):
    output_path = f"output_{len(output_audio_files)}.wav"
    clone.tts_to_file(text=text,
                      speaker_wav=voice,
                      language=language,
                      file_path=output_path)
    output_audio_files.append(output_path)

contents_to_read = []
def table_content(pdf_path):
    sentences_continued = ''
    pdf_document = fitz.open(pdf_path)

    # Process only the first 10 pages
    num_pages_to_process = min(10, pdf_document.page_count)

    for page_number in range(num_pages_to_process):
        page = pdf_document.load_page(page_number)

        text = page.get_text()
        text = re.sub(r'\s+', ' ', text)

        # Split text into sentences using NLTK's sent_tokenize
        sentences = nltk.sent_tokenize(text)

        # Count complete sentences to determine "normal" pages
        complete_sentence_count = sum(1 for sentence in sentences if sentence.strip() and re.search(r'\w', sentence.strip()))

        toc_keywords = ["Contents", "Table of Contents","Copyright","All rights reserved"]  # Define your TOC keywords here


        potential_toc_text = ' '.join(sentences)
        pdf_document.delete_page(page_number)
        for keyword in toc_keywords:
            if keyword in potential_toc_text:
                print(f"Page {page_number + 1} might contain the table of contents:")
                content_started = False
                for sentence in sentences:
                    if content_started and re.search(r'\w', sentence.strip()):
                        contents_to_read.append(sentence.strip())  # Add contents to the list
                    elif re.search(r'\b' + re.escape(keyword) + r'\b', sentence.strip()):
                        content_started = True
                    elif content_started and not re.search(r'\w', sentence.strip()):
                        break
                break

        pdf_document.delete_page(page_number)

    pdf_document.save("modified_document.pdf")
    pdf_document.close()
def extract_and_tokenize_text(pdf_path):
    sentences_continued = ''
    pdf_document = fitz.open(pdf_path)

    # Process only the first 10 pages
    num_pages_to_process = min(10, pdf_document.page_count)

    for page_number in range(num_pages_to_process):
        page = pdf_document.load_page(page_number)

        text = page.get_text()
        text = re.sub(r'\s+', ' ', text)

        # Split text into sentences using NLTK's sent_tokenize
        sentences = nltk.sent_tokenize(text)
        complete_sentence_count = sum(1 for sentence in sentences if sentence.strip() and re.search(r'\w', sentence.strip()))
        start_time = time.time()  # Record start time before processing the page

        for sentence in sentences:
            if sentence:
                Zero_Shot_CloneTTS(voice, language, sentence)

        end_time = time.time()  # Record end time after processing the page
        elapsed_time = end_time - start_time
        print(f"Page {page_number + 1} processed in {elapsed_time:.2f} seconds.")

    pdf_document.close()


for i in default_texts:
  Zero_Shot_CloneTTS(voice, language, i)



table_content(pdf_file_path)
# for k in contents_to_read:
#   Zero_Shot_CloneTTS(voice, language, k)

extract_and_tokenize_text("modified_document.pdf")


# Combine all generated audio files into a single file
combined_audio = AudioSegment.empty()
for audio_file in output_audio_files:
    audio_segment = AudioSegment.from_wav(audio_file)
    combined_audio += audio_segment

output_combined_audio = "combined_output.wav"
combined_audio.export(output_combined_audio, format="wav")
